In [1]:
import requests, json
import pandas as pd
from urllib.parse import quote
import xml.etree.ElementTree as ET
import numpy as np
import requests
from datetime import datetime, timedelta
import time

In [2]:
with open('keys/api.txt') as file:
    road_key = file.read()

In [17]:
base_url = f'http://openAPI.seoul.go.kr:8088/{road_key}/xml/DailyAverageAirQuality/1/5/20121225/'
gu_map = []
url = f'{base_url}{gu_map}'


In [20]:
url = f'http://openAPI.seoul.go.kr:8088/{road_key}/xml/DailyAverageAirQuality/1/5/20121225/강남구'

In [21]:
result = requests.get(url)
result.status_code

200

In [22]:
result.text

'<?xml version="1.0" encoding="UTF-8"?>\n<DailyAverageAirQuality>\n<list_total_count>1</list_total_count>\n<RESULT>\n<CODE>INFO-000</CODE>\n<MESSAGE>정상 처리되었습니다</MESSAGE>\n</RESULT>\n<row>\n<MSRDT_DE>20121225</MSRDT_DE>\n<MSRSTE_NM>강남구</MSRSTE_NM>\n<NO2>0.03</NO2>\n<O3>0.014</O3>\n<CO>0.3</CO>\n<SO2>0.006</SO2>\n<PM10>34</PM10>\n<PM25/>\n</row>\n</DailyAverageAirQuality>\n'

In [24]:
xml_data = result.text

In [25]:
# XML 파싱
root = ET.fromstring(xml_data)
rows = []

# 'row' 태그에 대한 모든 데이터 추출
for row in root.findall('.//row'):
    # 각 데이터 필드의 값을 추출하고, 데이터가 없으면 None (Pandas에서 NaN으로 변환됨) 으로 설정
    date = row.find('MSRDT_DE').text if row.find('MSRDT_DE') is not None else None
    station_name = row.find('MSRSTE_NM').text if row.find('MSRSTE_NM') is not None else None
    no2 = row.find('NO2').text if row.find('NO2') is not None else None
    o3 = row.find('O3').text if row.find('O3') is not None else None
    co = row.find('CO').text if row.find('CO') is not None else None
    so2 = row.find('SO2').text if row.find('SO2') is not None else None
    pm10 = row.find('PM10').text if row.find('PM10') is not None else None
    pm25 = row.find('PM25').text if row.find('PM25') is not None else None
    
    # 데이터프레임의 행으로 추가될 딕셔너리
    rows.append({
        "측정일시": date,
        "측정소명": station_name,
        "이산화질소": no2,
        "오존": o3,
        "일산화탄소": co,
        "아황산": so2,
        "미세": pm10,
        "초미세": pm25
    })

# Pandas DataFrame 생성
df = pd.DataFrame(rows)

# 데이터가 없는 경우 NaN으로 변환
df.replace({None: np.nan}, inplace=True)

print(df)

       측정일시 측정소명 이산화질소     오존 일산화탄소    아황산  미세  초미세
0  20121225  강남구  0.03  0.014   0.3  0.006  34  NaN


In [26]:
df.head(2)

,측정일시,측정소명,이산화질소,오존,일산화탄소,아황산,미세,초미세
0,20121225,강남구,0.03,0.014,0.3,0.006,34,NaN


In [30]:
# 구 이름 목록
gu_names = [
    "강남구", "강남대로", "강동구", "강변북로", "강북구", "강서구", "공항대로",
    "관악구", "광진구", "구로구", "금천구", "노원구", "도봉구", "도산대로",
    "동대문구", "동작구", "동작대로", "마포구", "서대문구", "서초구", "성동구",
    "성북구", "송파구", "신촌로", "양천구", "영등포구", "영등포로", "용산구",
    "은평구", "정릉로", "종로", "종로구", "중구", "중랑구", "천호대로",
    "청계천로", "한강대로", "홍릉로", "화랑로"
]


In [31]:
len(gu_names)

39

In [97]:

# API 키
with open('keys/api.txt') as file:
    road_key = file.read()

# 날짜 범위 설정
start_date = datetime(2023, 12, 1)
end_date = datetime(2023, 12, 8)
# 모든 데이터를 저장할 빈 데이터프레임 생성


for attempt in range(3):  # 최대 3번 재시도
    all_data_df1 = pd.DataFrame()
    try:

        # 지정된 날짜 범위에 대해 반복
        for single_date in (start_date + timedelta(n) for n in range((end_date - start_date).days + 1)):
            date_str = single_date.strftime("%Y%m%d")  # 날짜를 yyyymmdd 형태의 문자열로 변환
            
            # 각 구에 대한 API 요청을 보내고 결과를 처리합니다.
            for gu_name in gu_names:
                # API 요청 URL 생성
                url = f"http://openAPI.seoul.go.kr:8088/{road_key}/xml/DailyAverageAirQuality/1/5/{date_str}/{gu_name}"
                
                # API 요청
                result = requests.get(url, timeout=20)
                
                # API 요청 결과 처리
                if result.status_code == 200:
                    xml_data = result.text
                    # XML 파싱
                    root = ET.fromstring(xml_data)
                    rows = []

                    # 'row' 태그에 대한 모든 데이터 추출
                    for row in root.findall('.//row'):
                        # 각 데이터 필드의 값을 추출하고, 데이터가 없으면 None (Pandas에서 NaN으로 변환됨) 으로 설정
                        rows.append({
                            "측정일시": row.find('MSRDT_DE').text if row.find('MSRDT_DE') is not None else None,
                            "측정소명": row.find('MSRSTE_NM').text if row.find('MSRSTE_NM') is not None else None,
                            "이산화질소": row.find('NO2').text if row.find('NO2') is not None else None,
                            "오존": row.find('O3').text if row.find('O3') is not None else None,
                            "일산화탄소": row.find('CO').text if row.find('CO') is not None else None,
                            "아황산": row.find('SO2').text if row.find('SO2') is not None else None,
                            "미세": row.find('PM10').text if row.find('PM10') is not None else None,
                            "초미세": row.find('PM25').text if row.find('PM25') is not None else None
                        })

                    # 추출된 데이터로부터 DataFrame 생성
                    df = pd.DataFrame(rows)
                    
                    # 전체 데이터프레임에 추가
                    all_data_df1 = pd.concat([all_data_df1, df], ignore_index=True)
                else:
                    # 요청 실패 시 오류 메시지 출력
                    print(f"에러: {gu_name} - {date_str} - 상태 코드: {result.status_code}")
    except requests.exceptions.RequestException:
        time.sleep(5)  # 5초 대기 후 재시도

# 데이터가 없는 경우 NaN으로 변환
all_data_df1.replace({None: np.nan}, inplace=True)

In [98]:
all_data_df1.tail(1)

,측정일시,측정소명,이산화질소,오존,일산화탄소,아황산,미세,초미세
199,20231205,화랑로,0.0476,0.0038,0.69,0.0039,43,25


In [99]:
all_data_df1.to_csv('data/12월.csv', index=False)